# libs & config

In [1]:
# ajustando o caminho que o python vai procurar os arquivos
import os
import pandas as pd
import sys

# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

# PIPE

## 1 - Coleta e Identificação da Origem dos Dados (extract)

### 1.1 Recebimento do arquivo de origem & 1.2 Classificação da Planilha (Tipo e Finalidade)

In [49]:
from Scripts.Extract import ReadArchive, ConvertXml_toExcel, GetFile
# checa a apsta e se achar um arquivo, pega o path extençaõ para usar, SE NAO TIVER AS EXTENSAO ACEITA RETORNA ERRO
# ja esta fazendo o 2.3

found_excel_path = GetFile('xlsx')

if found_excel_path:  # vai direto
    file_path = found_excel_path
    df_prod = ReadArchive(file_path, 1)
else:
    found_excel_path = GetFile('xml')
    if found_excel_path:  # converte e depois le
        file_path = ConvertXml_toExcel(found_excel_path)
        df_prod = ReadArchive(file_path, 0)




Arquivo encontrado: d:\kaue\projetos\pipeline_estoque_bling_masquerade\Data\ESTOQUE MASQUERADE.XLSX


## 2. Classificação da Planilha (Tipo e Finalidade)


### 2.1 Tipo de produto & 2.2 Tipo de movimentação

## 4. 4. Transformação e Validação (Transform)
### 4.1 Carregar produtos já existentes no Bling via API

In [ ]:
from Scripts.Extract import GetProdutos, GetToken

token = GetToken() # precisa estar no arquivo .env
df_prod_bling = GetProdutos(token) # ja devolve em um DataFrame



In [ ]:
df_prod

# --  função PadronizeDataFrame
# 1. limpe os que nao fomos usar vazio e criar a coluna grade
# 2. transforme as colunas de tamnho em chaves de um diconario, apenas coluans que tiverem valor, adione na coluna grade
# 3. adicone os valores no diconario com base da scolunas que esta no  diconario, apois adicona na grade apaga as outras colunas
# 4. crie um produto(deve seguir o padro bling "cod,cor[setiver],tam") para cada chave da grade,e crie a coluna quant aond eela receb o respectivo valor da chave
# 5. veja quais produtos nao bate com o da bling, e crie uma coluna de status, se esta ok(true) ou nao(false)
# 6. compare com os produtos da bling, concertando codigos errados atravez de quanto for parecido o nome e cod se concertar atribui ok
# 7. compare novamente , se haver ainda dierentes cria um novo df para criar os produtos que nao existem na bling

# def PadronizeDataFrame(df_prod, df_prod_bling):
# 1. limpe os que nao fomos usar vazio e criar a coluna grade
df_prod.rename(columns={'total:': 'total'}, inplace=True)

tamanhos = ['U', 'PP', 'P', 'M', 'G', 'GG']

df_prod[tamanhos] = df_prod[tamanhos].astype(float)  # converte as colunas de tamanho para float, caso nao seja
df_prod["total"] = df_prod[tamanhos].sum(axis=1) # nao ta pegando da forula do excel
df_prod = df_prod.query('total != 0') # faz o papel do dropna
df_prod.reset_index(inplace=True)
df_prod['quant'] = ''
df_prod['CÓDIGO '].astype(str)

# lipando a coluna CÓDIGO, tirando os espaços e deixando padrao
for x in df_prod.index:
    old_code = df_prod.loc[x, 'CÓDIGO ']
    if not pd.isna(old_code):
        df_prod.loc[x,'CÓDIGO '] = old_code.strip().replace(" ", "0" if len(old_code) < 6 else "") # converte a coluna CÓDIGO para string, caso nao seja



new_x = len(df_prod) 
"""
!!!!! VER JEO=ITO CERTO ULIZANDO CONCAT !!!!!
"""

for x in df_prod.index:
    dict_grade = {}
    cod_prod = df_prod.loc[x, "CÓDIGO "]
    
    for tamanho in tamanhos:
        if not pd.isna(df_prod.loc[x, tamanho]): 
            dict_grade[tamanho] = df_prod.loc[x, tamanho]
    
    for grade, qtd in dict_grade.items():
        df_prod.loc[new_x, "CÓDIGO "] = f"{cod_prod}{grade}"
        df_prod.loc[new_x, "INFANTIL MENINOS"] = f"Tamanho:{grade}"
        df_prod.loc[new_x, "quant"] = qtd
        new_x += 1  



# 4. crie um produto(deve seguir o padro bling "cod,cor[setiver],tam") para cada chave da grade,e crie a coluna quant aond eela receb o respectivo valor da chave
  





In [52]:
df_prod.drop(columns=tamanhos, inplace=True)
df_prod.sort_values(by='CÓDIGO ', inplace=True)
df_prod.head(20)


,index,CÓDIGO,INFANTIL MENINOS,estoq. Min,total,quant
0,2.0,HW017,Capa Infantil Preta Bember 3/6 anos P,20.0,15.0,
53,NaN,HW017P,Tamanho:P,NaN,NaN,15.0
1,3.0,HW018,Capa Infantil Preta Bember 8/12 anos M,20.0,2.0,
54,NaN,HW018M,Tamanho:M,NaN,NaN,2.0
20,32.0,HW036,Bruxinha Jacy (Sem Chapéu),20.0,158.0,
107,NaN,HW036G,Tamanho:G,NaN,NaN,29.0
106,NaN,HW036M,Tamanho:M,NaN,NaN,87.0
105,NaN,HW036P,Tamanho:P,NaN,NaN,42.0
50,97.0,HW0592,Wizard (LANÇAMENTO) (tunica + capa + cinto),20.0,31.0,
189,NaN,HW0592U,Tamanho:U,NaN,NaN,31.0
